# Kaggle Notebook EDA

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import json

import scipy
import re
import difflib
from IPython.core.display import display, HTML
from operator import itemgetter
import copy
import dask
import dask.bag as db
import pickle

import ast

from random import sample

import pybktree
import editdistance
import tabulate
from tqdm.auto import tqdm

import pickle

from random import sample

import astor

from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

import sklearn

from diff_match_patch import diff_match_patch

I pulled all "kernels" related to the IEE Fraud Detection Kaggle competition using `python fetch_kaggle_notebooks.py ieee-fraud-detection ieee-fraud-detection --save_kernel_metadata`

In [5]:
def load_kaggle_kernel_json(path,scriptify=True):
    with open(path,"r") as file:
        result = json.loads(file.read())
        if result["kernelType"] == "notebook":
            source =  json.loads(result["source"])
            if scriptify:
                result["source"] = convert_nb_to_py_script(source) 
            else:
                result["source"] = source
        return result
  
    
def comment_cells(cell_source):
    if isinstance(cell_source,str):
        cell_source = cell_source.split("\n")
    lines = ["###"+x for x in cell_source]
    return "\n".join(lines)

def convert_nb_to_py_script(nb_json):
    result = []
    for cell in nb_json["cells"]:
        source = cell["source"]
        
        if isinstance(source,list):
            source = "\n".join(source)
        
        if cell["cell_type"] == "code":
            result.append(source)
        else:
            result.append(comment_cells(source))
    return "\n".join(result)

KERNEL_DIR = "/projects/bdata/kaggle/competitions/ieee-fraud-detection/"
kernel_json_paths = glob.glob(os.path.join(KERNEL_DIR,"*.json"))
kernel_jsons = []

In [6]:
for path in kernel_json_paths:
    res = load_kaggle_kernel_json(path)
    if res:
        kernel_jsons.append(res)
        
kernel_metadata = pd.read_csv(os.path.join(KERNEL_DIR,"metadata.csv"))

# Finding Similar Notebooks
## Levenshtein distance

In [3]:
def get_kernel_distances(kernels,metric):
    kernel_source = [x["source"] for x in kernels]
    print("hey")
    return pairwise_distances(kernel_source,metric)

    
def minimumEditDistance(s1,s2):
    if len(s1) > len(s2):
        s1,s2 = s2,s1
    distances = range(len(s1) + 1)
    for index2,char2 in enumerate(s2):
        newDistances = [index2+1]
        for index1,char1 in enumerate(s1):
            if char1 == char2:
                newDistances.append(distances[index1])
            else:
                newDistances.append(1 + min((distances[index1],
                                             distances[index1+1],
                                             newDistances[-1])))
        distances = newDistances
    return distances[-1]

### This is going to take too long :( 
Try tokenizing using Ge's method and use cosine similarity

## Cosine Similarity

In [7]:
def tokenize_code_snippet(code):
    try:
        no_chars = re.sub('[^a-zA-Z\n]+', ' ', code)
    except TypeError:
        print(code)
    tokens = split_func_name(no_chars)
    return tokens
    
def split_func_name(func):
    """
    split function names
    eg. sklearn.metrics.pairwise.cosine_similarity -> [sklearn, metrics, pairwise, cosine, similarity]
    """
    new_str = ''
    for i, l in enumerate(func):
        if i > 0 and l.isupper() and func[i - 1].islower():
            new_str += '.'
        elif i > 0 and i < len(func) - 1 and l.isupper() and func[i - 1].isupper() and func[i + 1].islower():
            new_str += '.'
        elif i > 0 and l.isdigit() and func[i - 1].isalpha():
            new_str += '.'
        elif i < len(func) - 1 and l.isalpha() and func[i - 1].isdigit():
            new_str += '.'
        else:
            pass
        new_str += l
    return re.split('\.|_|\s', new_str.lower())

def tokenize_kernels(kernels):
    return [tokenize_code_snippet(x["source"]) for x in kernels]

kernel_tokens = tokenize_kernels(kernel_jsons)
vectorizer = TfidfVectorizer(max_features=10000)
kernel_token_vectors = vectorizer.fit_transform([" ".join(x) for x in kernel_tokens])
normed_kernel_vectors = sklearn.preprocessing.normalize(kernel_token_vectors)

In [5]:
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(normed_kernel_vectors)
nn_distances, nn_indices = nbrs.kneighbors(normed_kernel_vectors)

/homes/gws/mikeam/anaconda3/envs/analysis/lib/python3.7/site-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [109]:
def html_diff(a,b):
    differ = difflib.HtmlDiff(wrapcolumn = 80)
    return differ.make_table(a.split("\n"),b.split("\n"), context = True)

def compare_diff(kernels, indices, print_description = True):
    orig = kernels[indices[0]]
    nn = kernels[indices[1]]
    if print_description:
        print("Left:",orig["author"], orig["slug"])
        print("Right:", nn["author"], nn["slug"])
    display(HTML(html_diff(orig["source"],nn["source"])))
    
def indices_to_author_slug(kernels, indices):
    to_return = []
    for i in indices:
        to_return.append([])
        for ii in i:
            to_return[-1].append(itemgetter("author","slug")(kernels[ii]))
    return to_return


Is the closest notebook usually uploaded by the same user?

In [7]:
nn_names = indices_to_author_slug(kernel_jsons,nn_indices)
sum([x[0][0] in  ([x[1][0],x[2][0]]) for x in nn_names])

152

Need to "align" these notebooks

## What about cells?

In [8]:
cell_jsons = []
for path in kernel_json_paths:
    res = load_kaggle_kernel_json(path,scriptify=False)
    if res:
        if res["kernelType"] != "notebook":
            continue
        kernel = []
        for cell in res["source"]["cells"]:
            if type(cell["source"]) is list:
                continue
            cell_json = copy.deepcopy(cell)
            cell_json.update({"kernel_id":res["id"]})
            cell_jsons.append(cell_json)

In [9]:
cell_tokens = tokenize_kernels(cell_jsons)
vectorizer = TfidfVectorizer(max_features=10000)
cell_token_vectors = vectorizer.fit_transform([" ".join(x) for x in cell_tokens])
normed_cell_vectors = sklearn.preprocessing.normalize(cell_token_vectors)

In [10]:
cell_nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(normed_cell_vectors)
cell_nn_distances, cell_nn_indices = cell_nbrs.kneighbors(normed_cell_vectors)

/homes/gws/mikeam/anaconda3/envs/analysis/lib/python3.7/site-packages/sklearn/neighbors/_base.py:413: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [11]:
compare_diff(cell_jsons, cell_nn_indices[500],print_description=False)

t,1,df_train = comb[:-a],t,1,df_train = comb[:len(df_train)]
,2,df_test = comb[-a:],,2,df_test = comb[len(df_train):]
,3,del comb,,3,del comb
,4,gc.collect(),,4,gc.collect()


In [12]:
compare_diff(cell_jsons, cell_nn_indices[130],print_description=False)

t,1,num_preds = 5,t,1,from keras.layers import concatenate
,2,num_features = test.shape[1],,,
,3,,,,
,4,,,,
,5,def get_model():,,,
,6,"inp = keras.layers.Input((num_features,))",,,
,7,"x = keras.layers.Reshape((num_features,1))(inp)",,,
,8,"x = keras.layers.Conv1D(128,num_preds, activation='elu')(x)",,,
,9,x = keras.layers.BatchNormalization()(x),,,
,10,"x = keras.layers.Conv1D(24,1, activation='elu')(x)",,,
,11,x = keras.layers.BatchNormalization()(x),,,


## What about a BK-Tree with Levenshtein distance?

In [13]:
editdistance.eval('banana', 'bahama')
tree = pybktree.BKTree(editdistance.eval, [x["source"] for x in cell_jsons])

In [14]:
def get_edit_BK_Tree(cells):
    cell_sources = [x["source"] for x in cell_jsons]
    tree = pybktree.BKTree(editdistance.eval, cell_sources)
    return tree

In [15]:
def display_matches(source,tree,edit_threshold = 25,limit = 10):
    matches = sorted(tree.find(source, edit_threshold))[:limit] 
#     matches.insert(0,"\n\n\n\n\n\n" +  matches[0][1]) 
#     print(matches)
    print(tabulate.tabulate(matches, headers = ["Edit Distance","Match"],tablefmt="fancy_grid"))

In [16]:
copy.deepcopy(tree)

<BKTree using eval with 1602 top-level nodes>

Differences are hyperparameters: 

In [17]:
query_string = """
NFOLDS = 5
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X.shape[0])
score = 0

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns
  
for fold_n, (train_index, valid_index) in enumerate(splits):
    X_train, X_valid = X[columns].iloc[train_index], X[columns].iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid)

    clf = lgb.train(params, dtrain, 10000, valid_sets = [dtrain, dvalid], verbose_eval=200, early_stopping_rounds=500)
    
    feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()
    
    y_pred_valid = clf.predict(X_valid)
    y_oof[valid_index] = y_pred_valid
    print(f"Fold {fold_n + 1} | AUC: {roc_auc_score(y_valid, y_pred_valid)}")
    
    score += roc_auc_score(y_valid, y_pred_valid) / NFOLDS
    y_preds += clf.predict(X_test) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()
    
print(f"\nMean AUC = {score}")
print(f"Out of folds AUC = {roc_auc_score(y, y_oof)}")"""
display_matches(query_string,tree, limit = 3)

╒═════════════════╤════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Edit Distance │ Match                                                                                                                  │
╞═════════════════╪════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│               4 │ NFOLDS = 15                                                                                                            │
│                 │ folds = KFold(n_splits=NFOLDS)                                                                                         │
│                 │                                                                                                                        │
│                 │ columns = X.columns                                                                                                    │
│            

In [18]:
query_string = """
for col in ['card1']: 
    valid_card = pd.concat([train_df[[col]], test_df[[col]]])
    valid_card = valid_card[col].value_counts()
    valid_card_std = valid_card.values.std()

    invalid_cards = valid_card[valid_card<=2]
    print('Rare cards',len(invalid_cards))

    valid_card = valid_card[valid_card>2]
    valid_card = list(valid_card.index)

    print('No intersection in Train', len(train_df[~train_df[col].isin(test_df[col])]))
    print('Intersection in Train', len(train_df[train_df[col].isin(test_df[col])]))
    
    train_df[col] = np.where(train_df[col].isin(test_df[col]), train_df[col], np.nan)
    test_df[col]  = np.where(test_df[col].isin(train_df[col]), test_df[col], np.nan)

    train_df[col] = np.where(train_df[col].isin(valid_card), train_df[col], np.nan)
    test_df[col]  = np.where(test_df[col].isin(valid_card), test_df[col], np.nan)
    print('#'*20)"""

display_matches(query_string,tree, edit_threshold=400, limit = 20)

╒═════════════════╤═══════════════════════════════════════════════════════════════════════════════════════════════════════╕
│   Edit Distance │ Match                                                                                                 │
╞═════════════════╪═══════════════════════════════════════════════════════════════════════════════════════════════════════╡
│             265 │ def siev_card_feature(df_train, df_test, col):                                                        │
│                 │     valid_card = pd.concat([df_train[[col]], df_test[[col]]])                                         │
│                 │     valid_card = valid_card[col].value_counts()                                                       │
│                 │     valid_card_std = valid_card.values.std()                                                          │
│                 │     invalid_cards = valid_card[valid_card < 2]                                                        │
│       

In this example there's a specific parameter that we're interested (`valid valid_card>2`) 

Maybe there's some way to match on graphs? 

In [19]:
# What are the closest matches for all cells?
metric_sources = lambda x,y: editdistance.eval(x["source"],y["source"])
cell_tree = pybktree.BKTree(metric_sources, cell_jsons) 

# Do this faster with Dask:

In [20]:
def get_nn(tree,cell,limit = 3, edit_threshold = 0.25, copy_tree=False):
    
    if edit_threshold < 1:
        edit_threshold = len(cell["source"]) / edit_threshold 
    if copy_tree:
        tree = copy.deepcopy(tree)
    nn = sorted(tree.find(cell, edit_threshold), key = lambda x: x[0])[:limit]
    return nn

all_nn = []
for cell in tqdm(sample(cell_jsons,2000)):
    nn = get_nn(cell_tree, cell, edit_threshold = 0.25)
    all_nn.append(nn)

KeyboardInterrupt: 

In [ ]:
pickle.dump(all_nn, open( "cell_nn.p", "wb" ))

In [ ]:
nn_sorted = sorted(all_nn,key = lambda x: x[1][0])
has_code_and_interesting = list(filter(lambda x: (x[0][1].get("cell_type") == "code") and x[1][0] >= 10,nn_sorted))

In [ ]:
def show_nn_diff(cell_nn):
    diff = html_diff(cell_nn[0][1]["source"], cell_nn[1][1]["source"])
    display(HTML(diff))

In [ ]:
show_nn_diff(has_code_and_interesting[10])

In [ ]:
show_nn_diff(has_code_and_interesting[230])

In [ ]:
show_nn_diff(has_code_and_interesting[300])

In [ ]:
show_nn_diff(has_code_and_interesting[150])

In [ ]:
show_nn_diff(has_code_and_interesting[250])

In [ ]:
show_nn_diff(has_code_and_interesting[71])

In [ ]:
show_nn_diff(has_code_and_interesting[85])

In [ ]:
show_nn_diff(has_code_and_interesting[18])

In [ ]:
is_long = list(filter(lambda x: len(x[0][1]["source"]) > 300, has_code_and_interesting))

In [ ]:
show_nn_diff(is_long[100])

# Mining Alternatives
Use Kaggle Tasks? https://www.kaggle.com/sobhanmoosavi/us-accidents/tasks?taskId=189
When is it important to understand the semantics of the code? Meaning in column names?

Another example: https://www.kaggle.com/datasnaek/youtube-new/tasks?taskId=258
## What might this look like?
Maybe it would be easier to define a decision point discretely examples from code\

For each row, what does the action look like?

| Stage    | Decision Point        | Original                                                         | Candidate Replacement                                                                                  |   |
|----------|-----------------------|------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------|---|
| Wrangle  | Missing Data          | `df = df.dropna()`                                               | `df = df.interpolate()`                                                                                |   |
| Wrangle  | Outliers              | `df = df[df.between(df.quantile(.15), df.quantile(.85))]`        | `df = df[df.between(df.quantile(.05), df.quantile(.95))]`                                               |   |
| Wrangle  | Filtering             | `df = df[df["completeness"] >= 0.9]`                             | `df = df[~df.groupby("date")["observation"].isnull().any()]`                                           |   |
| Wrangle  | Transform             | `df["x"] = np.log(df["x"])`                                      | `df["x"] = np.log(df["x"] + 1)`                                                                        |   |
| Wrangle  | Data Type             | `df["likert_response"] = pd.Categorical(df["likert_response"])`  | `df["likert_response"] = sklearn.preprocessing.OrdinalEncoder().fit_transform(df["likert_response"]))` |   |
| Model    | Assumption Violations | `stats.ttest_ind(rvs1, rvs2)`                                    | `stats.mannwhitneyu(rvs1, rvs2)`                                                                       |   |
| Model    | Operationalize DV     |                                                                |                                                                                                        |   |
| Model    | Operationalize IV     | `model = "risk ~ age + body_weight"`                             | `model = "risk ~ age + body_weight + is_smoker"`"                                                      |   |
| Model    | Covariates            | `model = "risk ~ age + body_weight"`                             | `model = "risk ~ age + body_weight + age*body_weight"`"                                                |   |
| Model    | Model                 | `smf.OLS.from_formula(model,data)`                               | `smf.GLM.from_formula(model,data)`                                                                     |   |
| Evaluate | Estimation Method     | `RegressionResults(model)`                                       | `RegressionResults(model).get_robust_results()`                                                        |   |
| Evaluate | Inference Criteria    | `RegressionResults(model)`                                       | `statsmodels.stats.multitest.multipletests(pvals,0.05)`                                          |   |

Could we do a skip-gram on blocks of data science code? Then autocomplete whatever we label as a "decision point" in the original source code? Maybe use a loss function that encourages relationships between notebooks with the same (or similar) tasks?

# Identifying Decision Points
If the first goal is to do this for one notebook, then is it enough to say we insert decision points wherever one of the above is used? If not, maybe we could do some kind of weak supervision (like in Coral) to label decision points according to a similar rubric. 

Maybe take a model like CORAL, add a decoder to it, and formulate the loss to encourage relationships between notebooks that we know to be related (either same competition or same task) 

# Using Versions


In [8]:
def parse_string(string):
    global c, d
    tree = ast.parse(string)
    
    json_tree = []
    def gen_identifier(identifier, node_type = 'identifier'):
        pos = len(json_tree)
        json_node = {}
        json_tree.append(json_node)
        json_node['type'] = node_type
        json_node['value'] = identifier
        return pos
    
    def traverse_list(l, node_type = 'list'):
        pos = len(json_tree)
        json_node = {}
        json_tree.append(json_node)
        json_node['type'] = node_type
        children = []
        for item in l:
            children.append(traverse(item))
        if (len(children) != 0):
            json_node['children'] = children
        return pos
        
    def traverse(node):
        pos = len(json_tree)
        json_node = {}
        json_tree.append(json_node)
        json_node['type'] = type(node).__name__
        children = []
        if isinstance(node, ast.Name):
            json_node['value'] = node.id
        elif isinstance(node, ast.Num):
            json_node['value'] = str(node.n)
        elif isinstance(node, ast.Str):
            json_node['value'] = node.s
        elif isinstance(node, ast.alias):
            json_node['value'] = str(node.name)
            if node.asname:
                children.append(gen_identifier(node.asname))
        elif isinstance(node, ast.FunctionDef):
            json_node['value'] = str(node.name)
        elif isinstance(node, ast.ClassDef):
            json_node['value'] = str(node.name)
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                json_node['value'] = str(node.module)
        elif isinstance(node, ast.Global):
            for n in node.names:
                children.append(gen_identifier(n))
        elif isinstance(node, ast.keyword):
            json_node['value'] = str(node.arg)
        

        # Process children.
        if isinstance(node, ast.For):
            children.append(traverse(node.target))
            children.append(traverse(node.iter))
            children.append(traverse_list(node.body, 'body'))
            if node.orelse:
                children.append(traverse_list(node.orelse, 'orelse'))
        elif isinstance(node, ast.If) or isinstance(node, ast.While):
            children.append(traverse(node.test))
            children.append(traverse_list(node.body, 'body'))
            if node.orelse:
                children.append(traverse_list(node.orelse, 'orelse'))
        elif isinstance(node, ast.With):
            children.append(traverse(node.context_expr))
            if node.optional_vars:
                children.append(traverse(node.optional_vars))
            children.append(traverse_list(node.body, 'body'))
        elif isinstance(node, ast.Try):
            children.append(traverse_list(node.body, 'body'))
            children.append(traverse_list(node.handlers, 'handlers'))
            if node.orelse:
                children.append(traverse_list(node.orelse, 'orelse'))
        elif isinstance(node, ast.arguments):
            children.append(traverse_list(node.args, 'args'))
            children.append(traverse_list(node.defaults, 'defaults'))
            if node.vararg:
                children.append(gen_identifier(node.vararg, 'vararg'))
            if node.kwarg:
                children.append(gen_identifier(node.kwarg, 'kwarg'))
        elif isinstance(node, ast.ExceptHandler):
            if node.type:
                children.append(traverse_list([node.type], 'type'))
            if node.name:
                children.append(traverse_list([node.name], 'name'))
            children.append(traverse_list(node.body, 'body'))
        elif isinstance(node, ast.ClassDef):
            children.append(traverse_list(node.bases, 'bases'))
            children.append(traverse_list(node.body, 'body'))
            children.append(traverse_list(node.decorator_list, 'decorator_list'))
        elif isinstance(node, ast.FunctionDef):
            children.append(traverse(node.args))
            children.append(traverse_list(node.body, 'body'))
            children.append(traverse_list(node.decorator_list, 'decorator_list'))
        else:
            # Default handling: iterate over children.
            for child in ast.iter_child_nodes(node):
                if isinstance(child, ast.expr_context) or isinstance(child, ast.operator) or isinstance(child, ast.boolop) or isinstance(child, ast.unaryop) or isinstance(child, ast.cmpop):
                    # Directly include expr_context, and operators into the type instead of creating a child.
                    json_node['type'] = json_node['type'] + type(child).__name__
                else:
                    children.append(traverse(child))
                
        if isinstance(node, ast.Attribute):
            children.append(gen_identifier(node.attr, 'attr'))
                
        if (len(children) != 0):
            json_node['children'] = children
        return pos
    
    traverse(tree)
    return json_tree

def get_param_from_filename(param,filename):
    template = "\?{}=(.*)\.|\?"
    query_regex = re.compile(template.format(param))
    try:
        return re.findall(query_regex,filename)[0]
    except IndexError:
        return None
    
def get_slug_from_file(filename):
    return re.split("\?|\.",filename)[0]

In [80]:
TEST_COMPETITION = "ieee-fraud-detection"
competition_submissions_path = os.path.join("../../data/processed/competitions_backup",TEST_COMPETITION)

def replace_function_subtrees(coral_repr):
    ignore = []
    new_tree = []
    for i in range(len(coral_repr)):
        node = coral_repr[i]
        if i in ignore:
            #ignore the children too:
            ignore = ignore + node.get("children",[])
            continue
        elif node["type"] == "Call":
            ignore = ignore + node.get("children",[])[1:]
        new_tree.append(node)
    return new_tree
            

class Cell(object):
    def __init__(self,slug,version_id,source):
        self.slug = slug
        self.version_id = version_id
        self.source = source
        self.coral_repr = parse_string(source)
        self.function_args_removed_repr = replace_function_subtrees(self.coral_repr)
        self.python_ast = ast.parse(source)
    
    def coral_diff(self,other,key = None,attr="coral_repr"):
        a_attr = getattr(self,attr)
        b_attr = getattr(other,attr)
        return self.tree_diff(a_attr, b_attr, key = key)
        
    def rear_pad_list(a,n):
        m = len(a)
        return a + [None for i in range(n-m)]
    
    def make_same_length(a,b):
        n = max(len(a),len(b))
    
        a = rear_pad_list(a,n)
        b = rear_pad_list(b,n)
    
        return (a,b)
    
    def tree_diff(self,a,b,key=None):
        a,b = make_same_length(a,b)
        if not key:
            key =  lambda aa,bb: not aa == bb
        return sum([key(aa,bb) for (aa,bb) in zip(a,b)])
    
    def __dict__(self):
        return {"slug":self.slug, "version_id" : self.version_id, 
               "source":self.source,"coral_repr":self.coral_repr}
    
kernel_trees = {}

#Could also try finding single lines


for version_path in tqdm(glob.glob(competition_submissions_path+ "/*.json")):
    filename = os.path.basename(version_path)
    version_id = get_param_from_filename("scriptVersionId",filename)
    if not version_id:
        continue
        
    slug = get_slug_from_file(filename)
    
    if not slug in kernel_trees:
        kernel_trees[slug] = {}
    
    with open(version_path) as kernel_file:
        try:
            res = json.load(kernel_file)
        except ValueError:
            continue 
            
        if not "cells" in res:
            continue
            
        version_cells = []
        
        for cell in res["cells"]:
            if type(cell["source"]) is list:
                continue
            
            try:
                version_cells.append(Cell(slug,version_id,cell["source"]))
                                                    
            except (SyntaxError, AttributeError):
                continue
        
        kernel_trees[slug][version_id] = version_cells
        
            

In [73]:
def rear_pad_list(a,n):
    m = len(a)
    return a + [None for i in range(n-m)]
    
def make_same_length(a,b):
    n = max(len(a),len(b))
    
    a = rear_pad_list(a,n)
    b = rear_pad_list(b,n)
    
    return (a,b)
    
def tree_diff(a,b,key=None):
    a,b = make_same_length(a,b)
    if not key:
        key =  lambda aa,bb: not aa == bb
    return sum([key(aa,bb) for (aa,bb) in zip(a,b)])

def looks_like_string(node):
    node_type = node.get("type")
    if node_type == "Constant":
        try:
            float(node.get("value"))
            return False
        except (ValueError,TypeError):
            return True
    else:
        return False
    
def dont_count_strings(a,b):
    if a is None or b is None:
        return True
    if looks_like_string(a) and looks_like_string(b):
        return False
    else:
        return (not a == b)
    
def get_matching_cells(kernel_trees,diff_versions = False, key = None,attr="coral_repr"):
    matches = []
    all_cells = []
    for slug,versions in tqdm(kernel_trees.items()):
        all_version_cells = []
        for version_id, cells in versions.items():
            if cells:
                for cell in cells:
                    all_version_cells.append(cell)

        n = len(all_version_cells)
        if n == 1:
            continue
        for i in range(n):
            for j in range(i+1,n):
                    
                cell_i = all_version_cells[i]
                cell_j = all_version_cells[j]
                
                if diff_versions:
                    if cell_i.version_id == cell_j.version_id:
                        continue
                diff = cell_i.coral_diff(cell_j,key=key,attr=attr)
                if diff == 1:
                    matches.append((cell_i,cell_j))
        all_cells = all_cells + all_version_cells
    return matches

In [107]:
matches = get_matching_cells(kernel_trees, diff_versions = True)

In [250]:
display(HTML(html_diff((matches[300][0].source),(matches[300][1].source))))

t,1,del raw_train_data,t,1,featuresToUse


In [83]:
def pprint_code_diff(a,b):
    dmp = diff_match_patch()
    diffs = dmp.diff_main(a,b)
    dmp.diff_cleanupSemantic(diffs)
    display(HTML(dmp.diff_prettyHtml(diffs)))


In [251]:
pprint_code_diff(matches[200][0].source,matches[200][1].source)

In [252]:
pprint_code_diff(matches[0][0].source,matches[0][1].source)

In [227]:
pprint_code_diff(matches[1000][0].source,matches[1000][1].source)

In [228]:
pprint_code_diff(matches[6100][0].source,matches[6100][1].source)

In [229]:
pprint_code_diff(matches[5900][0].source,matches[5900][1].source)

What is the distribution of these matches like?

In [157]:
slugs = [x[0].slug for x in matches]
pd.Series(slugs).value_counts()

ieee-cis-fraud-detection                           15316
kernel1a76f4bb95                                    9095
fraud-detection                                     7929
e-d-a-and-baseline-mix-lgbm                         7432
ieee-cis-competition-how-to-survive-the-shakeup     6000
                                                   ...  
lgbm-baseline-features-importance                      1
ieee-catboost-baseline-with-groupkfold-cv              1
flavour-of-autoencoder-0-0                             1
feature-engineered-lightgbm                            1
is-it-really-fraud                                     1
Length: 160, dtype: int64

In [231]:
matches_exclude_slug_bug = list(filter(lambda x: x[0].slug != "ieee-cis-fraud-detection", matches))

In [230]:
pprint_code_diff(matches_exclude_slug_bug[20][0].source,matches_exclude_slug_bug[20][1].source)

In [170]:
pprint_code_diff(matches_exclude_slug_bug[37][0].source,matches_exclude_slug_bug[37][1].source)

In [267]:
#Remove duplicates
def remove_duplicate_matches(matches):
    to_return = []
    record = set()
    for match in matches:
        if not (match[0].source,match[1].source) in record:
            record.add((match[0].source,match[1].source))
            to_return.append(match)
    return to_return
matches_no_dups = remove_duplicate_matches(matches)

In [177]:
pprint_code_diff(matches_no_dups[40][0].source,matches_no_dups[40][1].source)

In [178]:
pprint_code_diff(matches_no_dups[50][0].source,matches_no_dups[50][1].source)

In [179]:
pprint_code_diff(matches_no_dups[55][0].source,matches_no_dups[55][1].source)

In [180]:
pprint_code_diff(matches_no_dups[556][0].source,matches_no_dups[556][1].source)

In [181]:
pprint_code_diff(matches_no_dups[557][0].source,matches_no_dups[557][1].source)

In [182]:
pprint_code_diff(matches_no_dups[600][0].source,matches_no_dups[600][1].source)

In [183]:
pprint_code_diff(matches_no_dups[700][0].source,matches_no_dups[700][1].source)

In [185]:
pprint_code_diff(matches_no_dups[1000][0].source,matches_no_dups[1000][1].source)

In [186]:
pprint_code_diff(matches_no_dups[5000][0].source,matches_no_dups[5000][1].source)

## Let's try lines?

In [324]:
line_trees = {}

for slug,versions in tqdm(kernel_trees.items()):
    version_lines = []
    line_trees[slug] = {}
    for version_id, cells in versions.items():
        version_lines = []
        if cells:
            for cell in cells:
                for line in cell.source.split("\n"):
                    try:
                        version_lines.append(Cell(slug,version_id,line))
                    except SyntaxError:
                        continue
        line_trees[slug][version_id] = version_lines
    

# def get_matching_cells(kernel_trees):
#     matches = []
#     all_cells = []
#     for slug,versions in tqdm(kernel_trees.items()):
#         all_version_cells = []
#         for version_id, cells in versions.items():
#             if cells:
#                 for cell in cells:
#                     all_version_cells.append(cell)

#         n = len(all_version_cells)
#         if n == 1:
#             continue
#         for i in range(n):
#             for j in range(i+1,n):
#                 cell_i = all_version_cells[i]
#                 cell_j = all_version_cells[j]

#                 diff = cell_i.coral_diff(cell_j)
#                 if diff == 1:
#                     matches.append((cell_i,cell_j))
#         all_cells = all_cells + all_version_cells
#     return matches

In [212]:
line_matches = get_matching_cells(line_trees, diff_versions = True)

In [221]:
i = 8007
pprint_code_diff(line_matches[i][0].source,line_matches[i][1].source)

## Maybe don't count strings?
These tend to be too specific to the dataset IMO

In [19]:
matches_ignore_strings = get_matching_cells(kernel_trees, diff_versions = True, key = dont_count_strings)

In [300]:
matches_ignore_strings = remove_duplicate_matches(matches_ignore_strings)

In [301]:
pprint_code_diff(matches_ignore_strings[i][0].source,matches_ignore_strings[i][1].source)

IndexError: list index out of range

In [293]:
pprint_code_diff(matches_ignore_strings[20][0].source,matches_ignore_strings[20][1].source)

In [295]:
pprint_code_diff(matches_ignore_strings[1000][0].source,matches_ignore_strings[1000][1].source)

In [297]:
pprint_code_diff(matches_ignore_strings[9000][0].source,matches_ignore_strings[9000][1].source)

In [307]:
pprint_code_diff(matches_ignore_strings[1050][0].source,matches_ignore_strings[1050][1].source)

In [309]:
pprint_code_diff(matches_ignore_strings[2555][0].source,matches_ignore_strings[2555][1].source)

In [310]:
pprint_code_diff(matches_ignore_strings[1555][0].source,matches_ignore_strings[1555][1].source)

In [311]:
pprint_code_diff(matches_ignore_strings[155][0].source,matches_ignore_strings[155][1].source)

In [315]:
pprint_code_diff(matches_ignore_strings[322][0].source,matches_ignore_strings[322][1].source)

In [316]:
pprint_code_diff(matches_ignore_strings[422][0].source,matches_ignore_strings[422][1].source)

In [317]:
pprint_code_diff(matches_ignore_strings[450][0].source,matches_ignore_strings[450][1].source)

In [321]:
parse_string("metadata(train_trans[emailcard_col])")

[{'type': 'Module', 'children': [1]},
 {'type': 'Expr', 'children': [2]},
 {'type': 'Call', 'children': [3, 4]},
 {'type': 'NameLoad', 'value': 'metadata'},
 {'type': 'SubscriptLoad', 'children': [5, 6]},
 {'type': 'NameLoad', 'value': 'train_trans'},
 {'type': 'Index', 'children': [7]},
 {'type': 'NameLoad', 'value': 'emailcard_col'}]

In [322]:
parse_string("df = df.drop_duplicates()")

[{'type': 'Module', 'children': [1]},
 {'type': 'Assign', 'children': [2, 3]},
 {'type': 'NameStore', 'value': 'df'},
 {'type': 'Call', 'children': [4]},
 {'type': 'AttributeLoad', 'children': [5, 6]},
 {'type': 'NameLoad', 'value': 'df'},
 {'type': 'attr', 'value': 'drop_duplicates'}]

## Lines where we don't count strings?

In [7]:
line_matches_ignore_strings = get_matching_cells(line_trees, diff_versions = True, key = dont_count_strings)

NameError: name 'line_trees' is not defined

In [332]:
line_matches_ignore_strings = remove_duplicate_matches(line_matches_ignore_strings)

In [82]:
def display_random(k,matches):
    n = len(matches)
    indices = np.random.choice(range(n),k,replace = False)
    for i in indices:
        pprint_code_diff(matches[i][0].source,matches[i][1].source)

In [347]:
display_random(10, line_matches_ignore_strings)

In [353]:
display_random(10, line_matches_ignore_strings)

In [88]:
display_random(1,matches_ignore_strings)

NameError: name 'matches_ignore_strings' is not defined

In [376]:
display_random(6,matches_ignore_strings)

## Big Matches:

In [20]:
big_matches = list(filter(lambda x: len(x[0].source) > 100, matches_ignore_strings))

In [381]:
display_random(1,big_matches)

In [383]:
display_random(1,big_matches)

In [384]:
display_random(1,big_matches)

In [385]:
display_random(1,big_matches)

In [386]:
display_random(1,big_matches)

In [387]:
display_random(1,big_matches)

In [21]:
medium_matches = list(filter(lambda x: len(x[0].source) > 50 and len(x[0].source) < 100 , matches_ignore_strings))

In [391]:
display_random(1,medium_matches)

In [392]:
display_random(1,medium_matches)

In [393]:
display_random(1,medium_matches)

In [394]:
display_random(1,medium_matches)


In [396]:
display_random(5,medium_matches)

In [397]:
display_random(1,medium_matches)

In [398]:
display_random(1,medium_matches)

In [399]:
display_random(1,medium_matches)

In [400]:
display_random(1,medium_matches)

In [401]:
display_random(1,medium_matches)

In [111]:
for_ge = big_matches + medium_matches
for_ge_trees = [(x.source,y.source) for x,y in for_ge]
pickled_trees = pickle.dump(for_ge_trees,open("source_pairs.pickle","wb"))

In [113]:
for_ge_trees[0]

("EPOCHS = 5\ny_pred = np.zeros(sample_submission.shape[0])\ny_oof = np.zeros(X_train.shape[0])\nkf = KFold(n_splits = EPOCHS , shuffle = True)\nfor x_train_index , x_val_index in kf.split(X_train , Y_train):\n    clf = xgb.XGBClassifier(\n        n_estimators=500,\n        max_depth=4,\n        learning_rate=0.005,\n        subsample=0.2,\n        colsample_bytree = 0.2\n    )\n    x_tr , x_val = X_train.iloc[x_train_index , :] , X_train.iloc[x_val_index,:]\n    y_tr , y_val = Y_train.iloc[x_train_index] , Y_train.iloc[x_val_index]\n    clf.fit(x_tr,y_tr)\n    y_pred_train = clf.predict_proba(x_val)[:,1]\n    y_oof[x_val_index] = y_pred_train\n    print('ROC AUC {}'.format(roc_auc_score(y_val, y_pred_train)))\n    y_pred+= clf.predict_proba(X_test)[:,1] / EPOCHS\n\n ",
 "EPOCHS = 3\ny_pred = np.zeros(sample_submission.shape[0])\ny_oof = np.zeros(X_train.shape[0])\nkf = KFold(n_splits = EPOCHS , shuffle = True)\nfor x_train_index , x_val_index in kf.split(X_train , Y_train):\n    clf =

## Starting to look promising
- Maybe we oversample the large "alternatives"?
- Should also limiting ourselves to the "best" match for each cell 
      * (df = df.interpolate(), df = df.dropna()) or (df = df.interpolate(), training = df.interpolate())
- As per Tim's suggestion let's look at patterns that we're more familiar with:

In [131]:
def regex_search_matches(regex,matches):
    regex = re.compile(regex)
    to_return = []
    for a,b in matches:
        source_a = a.source
        source_b = b.source
        
        if re.match(regex,source_a) or re.match(regex,source_b):
            to_return.append((a,b))
    return to_return

In [104]:
has_logistic_regression = regex_search_matches(r".*transaction.*",matches)

In [110]:
print(len(has_logistic_regression))
display_random(1,has_logistic_regression)

13348


Sidebar: How many slugs are there?

In [75]:
sum([bool(x) for x in kernel_trees.values()]) 

568

## What if we don't count the arguments to functions? 

In [46]:
index=0

[print(x) for x in list(enumerate(list(list(kernel_trees.values())[index].values())[5][2].coral_repr))]

(0, {'type': 'Module', 'children': [1, 4, 8]})
(1, {'type': 'Assign', 'children': [2, 3]})
(2, {'type': 'NameStore', 'value': 'SEED'})
(3, {'type': 'Constant', 'value': '42'})
(4, {'type': 'Expr', 'children': [5]})
(5, {'type': 'Call', 'children': [6, 7]})
(6, {'type': 'NameLoad', 'value': 'seed_everything'})
(7, {'type': 'NameLoad', 'value': 'SEED'})
(8, {'type': 'Assign', 'children': [9, 10]})
(9, {'type': 'NameStore', 'value': 'LOCAL_TEST'})
(10, {'type': 'Constant'})


[None, None, None, None, None, None, None, None, None, None, None]

In [47]:
print(list(list(kernel_trees.values())[index].values())[5][2].source)

########################### Vars
#################################################################################
SEED = 42
seed_everything(SEED)
LOCAL_TEST = False


In [218]:
ignoring_subtrees = get_matching_cells(kernel_trees,attr="function_args_removed_repr", key = dont_count_strings)

In [76]:
test_tree = list(list(kernel_trees.values())[index].values())[5][2].coral_repr
replace_function_subtrees()

TypeError: replace_function_subtrees() missing 1 required positional argument: 'coral_repr'

In [219]:
display_random(1,ignoring_subtrees)

In [220]:
display_random(1,ignoring_subtrees)

In [221]:
display_random(1,ignoring_subtrees)

In [222]:
display_random(1,ignoring_subtrees)

In [223]:
display_random(1,ignoring_subtrees)

In [224]:
display_random(1,ignoring_subtrees)

In [225]:
display_random(1,ignoring_subtrees)

In [226]:
display_random(1,ignoring_subtrees)

In [227]:
display_random(1,ignoring_subtrees)

In [228]:
display_random(1,ignoring_subtrees)

In [268]:
large_no_subtrees = remove_duplicate_matches([x for x in ignoring_subtrees if len(x[0].source.split("\n")) > 5])

In [294]:
display_random(1,large_no_subtrees)

In [295]:
len(large_no_subtrees)

192

In [159]:
ignoring_subtrees[0][0].source

"EPOCHS = 5\ny_pred = np.zeros(sample_submission.shape[0])\ny_oof = np.zeros(X_train.shape[0])\nkf = KFold(n_splits = EPOCHS , shuffle = True)\nfor x_train_index , x_val_index in kf.split(X_train , Y_train):\n    clf = xgb.XGBClassifier(\n        n_estimators=500,\n        max_depth=4,\n        learning_rate=0.005,\n        subsample=0.2,\n        colsample_bytree = 0.2\n    )\n    x_tr , x_val = X_train.iloc[x_train_index , :] , X_train.iloc[x_val_index,:]\n    y_tr , y_val = Y_train.iloc[x_train_index] , Y_train.iloc[x_val_index]\n    clf.fit(x_tr,y_tr)\n    y_pred_train = clf.predict_proba(x_val)[:,1]\n    y_oof[x_val_index] = y_pred_train\n    print('ROC AUC {}'.format(roc_auc_score(y_val, y_pred_train)))\n    y_pred+= clf.predict_proba(X_test)[:,1] / EPOCHS\n\n "

In [208]:
display_random(1,has_logistic_regression)

In [130]:
display_random(1,has_logistic_regression)

In [132]:
display_random(1,has_logistic_regression)

In [133]:
display_random(1,has_logistic_regression)

In [134]:
display_random(1,has_logistic_regression)

In [297]:
list(kernel_trees.values())[index]

{'18867817': [],
 '18868525': [],
 '20704078': [],
 '18870049': [],
 '20703998': [],
 '18868150': [<__main__.Cell at 0x7f616b974130>,
 '18868180': []}